### Loading the Data

In [5]:
import os
from audio_image_pipeline import preprocess_to_hdf5

# absolute path because he somehow could not find the data otherwise idk 
# here we just have our folder that contains our audio data 
audio_folder = "/Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data/nsynth-valid/audio"

# this is the path where we save our converted data
output_h5_path = "/Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data/train_spectrograms_even.h5"

preprocess_to_hdf5(audio_folder, output_h5_path) # turns audio data into hdf5 data 

100%|██████████| 12678/12678 [01:01<00:00, 207.67it/s]

Preprocessed 12678 files to /Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data/train_spectrograms_even.h5


### Training

In [2]:
import torch
from torch.utils.data import DataLoader
from datasets import HDF5SpectrogramDataset
from nf_training_script import train_model  

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

train_data = HDF5SpectrogramDataset('/Users/koraygecimli/PycharmProjects/UDL_demo/wewantCNNs_project/data/train_spectrograms_even.h5')

train_loader = DataLoader( # again inspired by vae model 
    train_data,
    batch_size=32,  
    shuffle=True,
    pin_memory=True,
    num_workers=0,
    persistent_workers=False,
    prefetch_factor=None,
)

input_dim = train_data[0].numel()
trained_model = train_model(train_loader, input_dim=input_dim)

Using device: cpu


/Users/koraygecimli/PycharmProjects/UDL_demo/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/20, Loss: -87019.5103
Checkpoint saved at epoch 1


KeyboardInterrupt: 